In [1]:
from google.colab import drive
import os
import torch
import torchvision.transforms as transforms
from torchvision.utils import save_image
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import numpy as np
from diffusers import UNet2DModel, DDPMScheduler
from torch.cuda.amp import autocast

# Mount Google Drive
drive.mount('/content/drive')

# Set paths and categories
base_path = "/content/drive/MyDrive/DATASET"  # Path to your dataset
output_path = "/content/drive/MyDrive/NEWDATA"
os.makedirs(output_path, exist_ok=True)

# Define a mapping of image name prefixes to categories
category_map = {
    "indian": "Indians",
    "asian": "Orientals",
    "white": "Europeans"
}

# Step 2: Dataset Class to handle all images in one folder
class SingleFolderFaceDataset(Dataset):
    def __init__(self, base_dir, category_map, transform=None):
        self.transform = transform
        self.image_paths = []  # Will store tuples of (image_path, category)

        # Check if directory exists
        if not os.path.exists(base_dir):
            raise ValueError(f"Base directory not found: {base_dir}")

        # Collect images and categorize based on filename prefix
        for img_name in os.listdir(base_dir):
            if img_name.lower().endswith(('.jpg', '.jpeg', '.png')):
                img_path = os.path.join(base_dir, img_name)
                if os.path.isfile(img_path):
                    # Determine category from filename prefix
                    category = None
                    for prefix, category_name in category_map.items():
                        if img_name.lower().startswith(prefix):
                            category = category_name
                            break

                    if category:
                        self.image_paths.append((img_path, category))

        # Verify we found images
        if len(self.image_paths) == 0:
            raise ValueError(f"No valid images found in {base_dir}")

        print(f"Total images found: {len(self.image_paths)}")
        for category in category_map.values():
            category_count = sum(1 for path, cat in self.image_paths if cat == category)
            print(f"{category}: {category_count} images")

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path, category = self.image_paths[idx]
        try:
            image = Image.open(img_path).convert("RGB")
            if self.transform:
                image = self.transform(image)
            return image, category
        except Exception as e:
            print(f"Error loading image {img_path}: {str(e)}")
            return torch.zeros(3, 64, 64) if self.transform else Image.new('RGB', (64, 64), 'black'), category

# Print directory structure for debugging
print("Checking directory structure:")
print(f"Base path exists: {os.path.exists(base_path)}")
if os.path.exists(base_path):
    print("Contents of base directory:", os.listdir(base_path))

# Transform pipeline
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create dataset with error handling
try:
    dataset = SingleFolderFaceDataset(base_path, category_map, transform=transform)

    # Create dataloader
    data_loader = DataLoader(
        dataset,
        batch_size=8,
        shuffle=True,
        num_workers=0,
        drop_last=True
    )
    print(f"Successfully created DataLoader with {len(dataset)} total images")

except Exception as e:
    raise Exception(f"Error creating dataset/dataloader: {str(e)}")

# Step 3: Define Diffusion Model
model = UNet2DModel(
    sample_size=64,
    in_channels=3,
    out_channels=3,
    layers_per_block=2
)
scheduler = DDPMScheduler(num_train_timesteps=1000)

# Move model to GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")
model.to(device)

# Step 4: Training Loop
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-4)
num_epochs = 1000  # You can adjust this

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    batch_count = 0

    for batch, _ in data_loader:  # We don't need the category labels for training
        batch = batch.to(device)

        # Clear GPU memory
        torch.cuda.empty_cache()

        noise = torch.randn_like(batch)
        timesteps = torch.randint(0, scheduler.num_train_timesteps, (batch.size(0),), device=device).long()

        noisy_images = scheduler.add_noise(batch, noise, timesteps)
        predicted_noise = model(noisy_images, timesteps).sample

        loss = torch.nn.functional.mse_loss(predicted_noise, noise)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        batch_count += 1

    avg_loss = total_loss / batch_count
    print(f"Epoch {epoch + 1}/{num_epochs}, Average Loss: {avg_loss:.6f}")

# Step 5: Generate Images
model.eval()
combinations = [
    ("Orientals+Indians", "OI"),
    ("Orientals+Europeans", "OE"),
    ("Indians+Europeans", "IE"),
    ("Orientals+Indians+Europeans", "OIE")
]

for group_combo, group_label in combinations:
    combo_dir = os.path.join(output_path, group_label)
    os.makedirs(combo_dir, exist_ok=True)

    for i in range(10):
        noise = torch.randn(1, 3, 64, 64, device=device)
        timesteps = torch.arange(scheduler.num_train_timesteps - 1, -1, -1, device=device)

        with torch.no_grad():
            for t in timesteps:
                noisy_image = scheduler.add_noise(noise, noise, t)
                predicted_noise = model(noisy_image, t).sample
                noise = scheduler.step(predicted_noise, t, noisy_image).prev_sample

            generated_image = (noise.squeeze(0).cpu() + 1) / 2
            save_image(generated_image, f"{combo_dir}/{group_label}_{i + 1}.png")

        torch.cuda.empty_cache()
        print(f"Generated image {i+1} for combination {group_label}")

print("All images generated successfully!")

# Save the trained model
torch.save(model.state_dict(), os.path.join(output_path, 'diffusion_model.pth'))


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Mounted at /content/drive
Checking directory structure:
Base path exists: True
Contents of base directory: ['indian356.jpg', 'asian453.jpg', 'indian366.jpg', 'asian439.jpg', 'asian316.jpg', 'asian516.jpg', 'asian544.jpg', 'asian603.jpg', 'asian501.jpg', 'asian469.jpg', 'asian521.jpg', 'indian255.jpg', 'asian569.jpg', 'asian262.jpg', 'indian392.jpg', 'indian268.jpg', 'asian689.jpg', 'indian270.jpg', 'indian294.jpg', 'asian359.jpg', 'indian462.jpg', 'asian622.jpg', 'asian614.jpg', 'indian253.jpg', 'asian377.jpg', 'indian426.jpg', 'indian260.jpg', 'indian279.jpg', 'asian495.jpg', 'indian306.jpg', 'asian353.jpg', 'indian184.jpg', 'asian349.jpg', 'indian281.jpg', 'indian273.jpg', 'indian358.jpg', 'asian334.jpg', 'indian390.jpg', 'indian263.jpg', 'asian297.jpg', 'indian416.jpg', 'asian485.jpg', 'asian456.jpg', 'indian185.jpg', 'asian601.jpg', 'asian478.jpg', 'indian463.jpg', 'asian457.jpg', 'asian232.jpg', 'asian554.jpg', 'indian208.jpg', 'asian298.jpg', 'asian264.jpg', 'indian407.jpg', 'asi

/usr/local/lib/python3.10/dist-packages/diffusers/configuration_utils.py:140: FutureWarning: Accessing config attribute `num_train_timesteps` directly via 'DDPMScheduler' object attribute is deprecated. Please access 'num_train_timesteps' over 'DDPMScheduler's config object instead, e.g. 'scheduler.config.num_train_timesteps'.
  deprecate("direct config name access", "1.0.0", deprecation_message, standard_warn=False)


Epoch 1/20, Average Loss: 1.003746
Epoch 2/20, Average Loss: 0.997717
Epoch 3/20, Average Loss: 0.993895
Epoch 4/20, Average Loss: 0.991025
Epoch 5/20, Average Loss: 0.986799
Epoch 6/20, Average Loss: 0.982772
Epoch 7/20, Average Loss: 0.982338
Epoch 8/20, Average Loss: 0.980825
Epoch 9/20, Average Loss: 0.979672
Epoch 10/20, Average Loss: 1.021696
Epoch 11/20, Average Loss: 0.999701
Epoch 12/20, Average Loss: 1.000263
Epoch 13/20, Average Loss: 1.000161
Epoch 14/20, Average Loss: 1.000339
Epoch 15/20, Average Loss: 0.999937
Epoch 16/20, Average Loss: 1.000034
Epoch 17/20, Average Loss: 0.999456
Epoch 18/20, Average Loss: 1.000554
Epoch 19/20, Average Loss: 1.000330
Epoch 20/20, Average Loss: 0.999598
Generated image 1 for combination OI
Generated image 2 for combination OI
Generated image 3 for combination OI
Generated image 4 for combination OI
Generated image 5 for combination OI
Generated image 6 for combination OI
Generated image 7 for combination OI
Generated image 8 for combinat